In [1]:
import sys
import os

# Add the parent directory to the system path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../INTENT RECOGNITION')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../NAME ENTITY RECOGNITION')))


In [2]:
from ner import NamedEntityRecognition

In [3]:
from intent_recognition import IntentRecognition

In [4]:
train_config = { 'use_class_weights':True}
hyperparams = {'vocab_size': 500, 'embedding_dim': 768, 'epochs': 10, 'batch_size': 32}

In [5]:
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, LSTM, GRU, Bidirectional, Conv1D, GlobalMaxPooling1D, GlobalAveragePooling1D, Dense, Dropout


In [6]:
model = Sequential()
model.add(Bidirectional(LSTM(64, return_sequences=True), merge_mode="ave"))
model.add(GlobalMaxPooling1D())

basic = IntentRecognition(
            model, 
            hyperparams, 
            training_times=1, 
            train_config=train_config, 
            verbosing=0, 
            name="Bidirectional",
            save_results=False
        )
basic.train_model()

Training model 1/1



In [7]:
tokenized = np.array(basic.tokenizer.texts_to_sequences(["I want a to eat a steak to Boston please"]))

In [8]:
probs = basic.model.predict(tokenized)
_predicted_labels = np.argmax(probs, axis=1)
predicted_labels = basic.label_encoder.inverse_transform(_predicted_labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


In [9]:
predicted_labels

array(['flight'], dtype='<U26')

In [10]:
model = Sequential()
model.add(Bidirectional(GRU(128, return_sequences=True)))

ner_model = NamedEntityRecognition(
            model, 
            hyperparams, 
            training_times=1, 
            train_config=train_config, 
            verbosing=0, 
            name="Bidirectional",
            save_results=False
        )
ner_model.train_model()

29/29 ━━━━━━━━━━━━━━━━━━━━ 13s 189ms/step
897
897
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step


FileNotFoundError: [Errno 2] No such file or directory: './results/NER_results_complete.csv'

In [19]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [31]:
tokenized2 = np.array(ner_model.tokenizer.texts_to_sequences(["i need a ticket to go to san diego"]))

tokenized2

array([[ 12,  55,  14, 171,   1,  78,   1,  11, 105]])

In [33]:
max_length = ner_model.model.input_shape[1]  # Adjust based on your model’s expected input
tokenized2_padded = pad_sequences(tokenized2, maxlen=max_length, padding="post")

In [35]:
probs = ner_model.model.predict(tokenized2_padded)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step


In [36]:
probs = ner_model.model.predict(tokenized2_padded)
_predicted_labels = np.argmax(probs, axis=2)
_predicted_labels
predicted_labels = [ner_model.label_encoder.inverse_transform(a) for a in _predicted_labels]
print(predicted_labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step
[array(['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-toloc.city_name',
       'I-toloc.city_name', 'B-toloc.state_name', 'I-mod', '<pad>',
       '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>',
       '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>',
       '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>',
       '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>',
       '<pad>', '<pad>', '<pad>', '<pad>', '<pad>'], dtype='<U28')]


In [ ]:
# Import necessary packages
import numpy as np

class DialogSystem:
    def __init__(self, intent_model, ner_model):
        """
        Initialize the DialogSystem with intent recognition and named entity recognition models.
        
        Parameters:
        - intent_model: an instance of IntentRecognition trained model.
        - ner_model: an instance of NamedEntityRecognition trained model.
        """
        self.intent_model = intent_model
        self.ner_model = ner_model
    
    def preprocess_input(self, user_input):
        """
        Preprocess the user input for tokenization.
        """
        return np.array(self.intent_model.tokenizer.texts_to_sequences([user_input]))

    def get_intent(self, user_input):
        """
        Predict the intent of the user input using the intent model.
        """
        tokenized_input = self.preprocess_input(user_input)
        tokenized2_padded = pad_sequences(tokenized_input, maxlen=max_length, padding="post")
        probs = self.intent_model.model.predict(tokenized2_padded)
        predicted_label = np.argmax(probs, axis=1)
        return self.intent_model.label_encoder.inverse_transform(predicted_label)[0]

    def get_entities(self, user_input):
        """
        Extract entities from the user input using the NER model.
        """
        tokenized_input = np.array(self.ner_model.tokenizer.texts_to_sequences([user_input]))
        max_length = self.ner_model.model.input_shape[1]  # Adjust based on your model’s expected input
        tokenized2_padded = pad_sequences(tokenized_input, maxlen=max_length, padding="post")
        probs = self.ner_model.model.predict(tokenized2_padded)
        predicted_labels = np.argmax(probs, axis=2)[0]
        entities = []
        current_entity = None
        for i, label in enumerate(predicted_labels[:len(user_input.split())]):
            label_str = self.ner_model.label_encoder.inverse_transform([label])[0]
            if label_str != 'O':  # Ignore 'O' (Outside) labels
                if label_str.startswith("B-"):  # Start of a new entity
                    if current_entity:
                        entities.append(current_entity)  # Add the previous entity
                    current_entity = {"entity": label_str, "text": user_input.split()[i]}  # Initialize a new entity
                elif label_str.startswith("I-"):  # Inside an existing entity
                    if current_entity:
                        current_entity["text"] += " " + user_input.split()[i]  # Add the token to the current entity
            else:
                if current_entity:
                    entities.append(current_entity)  # End of an entity
                    current_entity = None
        
        if current_entity:  # Add the last entity
            entities.append(current_entity)

        return entities

    def generate_response(self, intent, entities):
        """
        Generate a response based on the identified intent and entities.
        """
        city_name = None
        state_name = None
        
        # Look for city and state names in the entities
        for entity in entities:
            if "city_name" in entity["entity"]:
                city_name = entity["text"]
            elif "state_name" in entity["entity"]:
                state_name = entity["text"]
        
        if intent == "flight":
            if city_name:
                return f"Looking up flight information to {city_name}."
            elif state_name:
                return f"Looking up flight information to {state_name}."
            else:
                return "Please specify a destination for the flight."

        elif intent == "airline":
            return "Retrieving airline information for you."

        elif intent == "airport":
            if city_name or state_name:
                location = city_name if city_name else state_name
                return f"Looking up airport information near {location}."
            else:
                return "Please specify a location for the airport."

        else:
            return "I'm sorry, I didn't understand your request."

    def handle_user_input(self, user_input):
        """
        Process user input, identify intent and entities, and generate a response.
        """
        intent = self.get_intent(user_input)
        entities = self.get_entities(user_input)
        response = self.generate_response(intent, entities)
        return response

# Initialize the dialog system with the intent and NER models
dialog_system = DialogSystem(basic, ner_model)

# Define a function to interact in the notebook
def chat_with_bot():
    print("Welcome to the ATIS-based dialog system! Type 'exit' to end the conversation.")
    while True:
        user_input = input("You: ")
        print("You: ", user_input)
        if user_input.lower() == 'exit':
            print("Bot: Goodbye!")
            break
        response = dialog_system.handle_user_input(user_input)
        print("Bot:", response)

# Run the dialog system
chat_with_bot()


Welcome to the ATIS-based dialog system! Type 'exit' to end the conversation.
You:  information about airports in california
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Bot: Looking up flight information to california.
You:  california
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Bot: Looking up flight information to california.
You:  


ValueError: Exception encountered when calling LSTM.call().

[1mslice index 0 of dimension 1 out of bounds. for '{{node sequential_1_1/bidirectional_1/forward_lstm_1/strided_slice}} = StridedSlice[Index=DT_INT32, T=DT_FLOAT, begin_mask=5, ellipsis_mask=0, end_mask=5, new_axis_mask=0, shrink_axis_mask=2](sequential_1_1/embedding_1/GatherV2, sequential_1_1/bidirectional_1/forward_lstm_1/strided_slice/stack, sequential_1_1/bidirectional_1/forward_lstm_1/strided_slice/stack_1, sequential_1_1/bidirectional_1/forward_lstm_1/strided_slice/stack_2)' with input shapes: [1,0,768], [3], [3], [3] and with computed input tensors: input[1] = <0 0 0>, input[2] = <0 1 0>, input[3] = <1 1 1>.[0m

Arguments received by LSTM.call():
  • sequences=tf.Tensor(shape=(1, 0, 768), dtype=float32)
  • initial_state=None
  • mask=None
  • training=False

In [ ]:
# Define a function to interact in the notebook
def chat_with_bot():
    print("Welcome to the dialog system! Type 'exit' to end the conversation.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            print("Bot: Goodbye!")
            break
        response = dialog_system.handle_user_input(user_input)
        print("Bot:", response)

# Run the dialog system
chat_with_bot()